In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as plt


In [2]:
df = pd.read_csv("dota_train.csv")
df_test = pd.read_csv("dota_test.csv")
df.head()
df.info(verbose=True)
df.isnull().sum()


C:\Users\Василий\AppData\Local\Temp\ipykernel_16408\3913565053.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dota_train.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409794 entries, 0 to 409793
Data columns (total 103 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    match_id                     int64  
 1    r1_hero                      int64  
 2    r1_level                     int64  
 3    r1_xp                        int64  
 4    r1_gold                      int64  
 5    r1_lh                        int64  
 6    r1_kills                     int64  
 7    r1_deaths                    int64  
 8    r1_items                     int64  
 9    r2_hero                      int64  
 10   r2_level                     int64  
 11   r2_xp                        int64  
 12   r2_gold                      int64  
 13   r2_lh                        int64  
 14   r2_kills                     int64  
 15   r2_deaths                    int64  
 16   r2_items                     int64  
 17   r3_hero                      int64  
 18   r3_level              

match_id                        0
r1_hero                         0
r1_level                        0
r1_xp                           0
r1_gold                         0
                            ...  
dire_ward_observer_count        0
dire_ward_sentry_count          0
dire_first_ward_time        43843
duration                        0
radiant_win                     8
Length: 103, dtype: int64

In [5]:
df[['first_blood_time', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time']] = df[['first_blood_time', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time']].abs()
df[['dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']] = df[['dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']].abs()

In [3]:
def CounT(X_):

    #X_ = X_.fillna(0)
    X_ = X_.fillna(X_.mean())
    #X_ = X_.fillna(X_.median())
    X_.drop(labels=["match_id"], axis=1, inplace=True)

    X_cat = X_[["r1_hero","r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"]]
    ind = [] # Список индексов героев, которых использовали хотя бы 1 раз за все время
    ind_no = [] # Которых не использовали вообще

    for i in range(1,130):
        if sum((X_cat==i).sum()) > 0:
            ind.append(i)
        else:
            ind_no.append(i)
    N = max(ind)
    X_pick = np.zeros((X_cat.shape[0], N))
    for index, row in X_cat.iterrows():
        p=0
        for j in row:
            if p < 5:
                X_pick[index][int(j)-1] = 1
                p+=1
            else:
                X_pick[index][int(j)-1] = -1
    X_pick_df = pd.DataFrame(X_pick)
    col = ['%d_hero' % (i+1) for i in range(N)]

    X_pick_df=X_pick_df.set_axis(col, axis=1)

    for i in ind_no:
        if i<=N:
            X_pick_df = X_pick_df.drop(labels=["%d_hero" % i], axis=1)
    X_without_categories = X_.drop(labels=["r1_hero","r2_hero","r3_hero","r4_hero","r5_hero","d1_hero","d2_hero","d3_hero","d4_hero","d5_hero"], axis=1)
    X_new = pd.concat([X_without_categories, X_pick_df], axis=1)
    return X_new

In [4]:
match_id = df_test["match_id"]
df = CounT(df)
df_test = CounT(df_test)

In [5]:
X = df.drop('radiant_win', axis=1)
y = df['radiant_win']
y = y.astype(int)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
for col in X.columns:
    print(X[col].describe())
#sb.boxplot(X_train['dire_first_ward_time'])
#for col in X_train.columns:
#    sb.boxplot(X_train[col])

count    409794.000000
mean          0.625807
std           0.786031
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           5.000000
Name: r1_level, dtype: float64
count    409794.000000
mean       1267.234450
std         505.016573
min           0.000000
25%         894.000000
50%        1180.000000
75%        1648.000000
max        3470.000000
Name: r1_xp, dtype: float64
count    409794.000000
mean       1148.777388
std         428.227612
min           0.000000
25%         817.000000
50%        1105.000000
75%        1421.000000
max        4616.000000
Name: r1_gold, dtype: float64
count    409794.000000
mean         10.852206
std           8.041734
min           0.000000
25%           4.000000
50%          10.000000
75%          17.000000
max          51.000000
Name: r1_lh, dtype: float64
count    409794.000000
mean          0.393354
std           0.695038
min           0.000000
25%           0.000000
50%           0.000000
75%      

In [10]:
import seaborn as sb

X['target'] = y
X['first_blood_team']=X['first_blood_team'].astype(int)
cols = X.columns[X.dtypes != 'object']

corr = X[cols].drop('first_blood_team', axis=1).corr()
#corr = corr[abs(corr) > 0.8].stack().reset_index()
#corr = corr[corr['level_0'] != corr['level_1']]
print(corr)
#sb.heatmap(corr, cmap="Blues", annot=True)

#X.drop('target', axis=1, inplace=True)

          r1_level     r1_xp   r1_gold     r1_lh  r1_kills  r1_deaths  \
r1_level  1.000000  0.146629  0.062127  0.063736 -0.004290  -0.013588   
r1_xp     0.146629  1.000000  0.686848  0.661990  0.150847  -0.105724   
r1_gold   0.062127  0.686848  1.000000  0.788715  0.456777  -0.092719   
r1_lh     0.063736  0.661990  0.788715  1.000000 -0.016341  -0.105355   
r1_kills -0.004290  0.150847  0.456777 -0.016341  1.000000  -0.006710   
...            ...       ...       ...       ...       ...        ...   
109_hero -0.001331  0.000509 -0.000478  0.001760 -0.002032   0.009789   
110_hero -0.001110 -0.006990 -0.012634 -0.009045 -0.007886  -0.000756   
111_hero  0.000581  0.001543  0.001555  0.000910  0.001449   0.006437   
112_hero -0.000792 -0.001311  0.000354 -0.002906  0.003969   0.001883   
target   -0.000473  0.044341  0.072738  0.038961  0.056435  -0.353802   

          r1_items  r2_level     r2_xp   r2_gold  ...  103_hero  104_hero  \
r1_level -0.076135  0.743342 -0.019916 -0.0506

In [29]:
def find_and_remove_outliers(train_df, k=1.5):
    bounds = {}
    for col in train_df.select_dtypes(include=['int64', 'float64']).columns:
        Q1 = train_df[col].quantile(0.25)
        Q3 = train_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - k * IQR
        upper = Q3 + k * IQR
        bounds[col] = {'lower': lower, 'upper': upper}
        train_df = train_df[(train_df[col] >= lower) & (train_df[col] <= upper)]
    return train_df, bounds
X_trn = pd.concat([X_train, y_train], axis=1)
df_train, bounds = find_and_remove_outliers(X_trn, k=1.5)
X_train = df_train.drop('radiant_win', axis=1)
y_train = df_train['radiant_win']

r1_level r1_level  r1_xp  r1_gold  r1_lh  r1_kills  r1_deaths  r1_items  r2_level  r2_xp  r2_gold  r2_lh  r2_kills  r2_deaths  r2_items  r3_level  r3_xp  r3_gold  r3_lh  r3_kills  r3_deaths  r3_items  r4_level  r4_xp  r4_gold  r4_lh  r4_kills  r4_deaths  r4_items  r5_level  r5_xp  r5_gold  r5_lh  r5_kills  r5_deaths  r5_items  d1_level  d1_xp  d1_gold  d1_lh  d1_kills  d1_deaths  d1_items  d2_level  d2_xp  d2_gold  d2_lh  d2_kills  d2_deaths  d2_items  d3_level  d3_xp  d3_gold  d3_lh  d3_kills  d3_deaths  d3_items  d4_level  d4_xp  d4_gold  d4_lh  d4_kills  d4_deaths  d4_items  d5_level  d5_xp  d5_gold  d5_lh  d5_kills  d5_deaths  d5_items  first_blood_time  first_blood_team  first_blood_player1  first_blood_player2  radiant_bottle_time  radiant_courier_time  radiant_flying_courier_time  radiant_tpscroll_count  radiant_boots_count  radiant_ward_observer_count  radiant_ward_sentry_count  radiant_first_ward_time  dire_bottle_time  dire_courier_time  dire_flying_courier_time  dire_tpscrol

In [17]:
def apply_bounds_to_test(test_df, bounds):
    for col in bounds:
        test_df = test_df[
            (test_df[col] >= bounds[col]['lower']) &
            (test_df[col] <= bounds[col]['upper'])
        ]
    return test_df
X_tst = pd.concat([X_test, y_test], axis=1)
df_test = apply_bounds_to_test(X_tst, bounds)
X_test = df_test.drop('radiant_win', axis=1)
y_test = df_test['radiant_win']

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR, _libsvm
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [12]:
p1 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(penalty='l2', solver='lbfgs', random_state=42))
])
p2 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=42))
])
p3 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('model', LinearSVC(penalty='l2', random_state=42))
])
p4 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('model', SGDClassifier(loss='log_loss', alpha=0.0001, penalty='l2', l1_ratio=0.5))
])
p5 = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LogisticRegression(penalty='l2', solver='lbfgs', random_state=42))
])
p6 = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=42))
])
p7 = Pipeline([
    ('scaler', MinMaxScaler()),
    ('pca', PCA(n_components=10)),
    ('model', LinearSVC(penalty='l2', random_state=42))
])
p8 = Pipeline([
    ('scaler', MinMaxScaler()),
    ('pca', PCA(n_components=10)),
    ('model', SGDClassifier(loss='log_loss', alpha=0.0001, penalty='l2', l1_ratio=0.5))
])
# p4 = Pipeline([
#     ('scaler', StandardScaler()),
#     ('feature_selector', SelectKBest(score_func=f_classif, k=10)),  # Выбор 10 лучших признаков
#     ('model', LogisticRegression(penalty='l2', solver='lbfgs', random_state=42))
# ])
p9 = Pipeline([
    ('model', XGBClassifier())
])
p10 = Pipeline([
    ('model', CatBoostClassifier())
])
p11 = Pipeline([
    ('model', LGBMClassifier())
])
p12 = Pipeline([
    ('model', RandomForestClassifier(n_estimators=10, class_weight='balanced'))
])
p13 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('model', VotingClassifier([
        ('rf', RandomForestClassifier(n_estimators=10)),
        ('xgb', XGBClassifier(loss='log_loss', alpha=0.0001, penalty='l2', l1_ratio=0.5))
    ]))
])


In [13]:
p14 = Pipeline([
    ('model', RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced'))
])
p15 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=30)),
    ('model', VotingClassifier([
        ('lg', LogisticRegression(penalty='l2', solver='lbfgs', random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced')),
        ('xgb', XGBClassifier(loss='log_loss', alpha=0.0001, penalty='l2', l1_ratio=0.5))
    ]))
])

In [14]:
#p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13
for i,p in enumerate([p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15]):
    p.fit(X_train, y_train)
    pred = p.predict(X_test)
    print(i+1, roc_auc_score(y_test, pred))


1 0.8936051312041707
2 0.8441712187987136
3 0.6298524278643232
4 0.6338411749876294


C:\Users\Василий\PycharmProjects\pythonProject2\venv\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


5 0.8934378996411318
6 0.8441712187987136
7 0.5565890728395311
8 0.5583752020274175
9 0.8905200640212504
Learning rate set to 0.118891
0:	learn: 0.6196110	total: 218ms	remaining: 3m 37s
1:	learn: 0.5619706	total: 300ms	remaining: 2m 29s
2:	learn: 0.5192231	total: 376ms	remaining: 2m 4s
3:	learn: 0.4854165	total: 455ms	remaining: 1m 53s
4:	learn: 0.4592005	total: 526ms	remaining: 1m 44s
5:	learn: 0.4381504	total: 598ms	remaining: 1m 39s
6:	learn: 0.4209054	total: 667ms	remaining: 1m 34s
7:	learn: 0.4064951	total: 737ms	remaining: 1m 31s
8:	learn: 0.3947257	total: 806ms	remaining: 1m 28s
9:	learn: 0.3845327	total: 869ms	remaining: 1m 26s
10:	learn: 0.3757644	total: 932ms	remaining: 1m 23s
11:	learn: 0.3676649	total: 999ms	remaining: 1m 22s
12:	learn: 0.3610332	total: 1.06s	remaining: 1m 20s
13:	learn: 0.3550514	total: 1.13s	remaining: 1m 19s
14:	learn: 0.3496408	total: 1.19s	remaining: 1m 18s
15:	learn: 0.3451142	total: 1.26s	remaining: 1m 17s
16:	learn: 0.3410049	total: 1.32s	remaining:

C:\Users\Василий\PycharmProjects\pythonProject2\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [20:14:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "l1_ratio", "loss", "penalty" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


13 0.6375510248084441
14 0.8628304208832871


C:\Users\Василий\PycharmProjects\pythonProject2\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [20:18:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "l1_ratio", "loss", "penalty" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


15 0.8443241409247411


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from scipy.stats import loguniform
from scipy.stats import uniform

1 5 9 10 11 14 15

In [8]:
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 10)
}

model = RandomForestClassifier(random_state=42)

grid_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая точность:", grid_search.best_score_)

best_model = grid_search.best_estimator_

Лучшие параметры: {'max_depth': 14, 'min_samples_split': 5, 'n_estimators': 182}
Лучшая точность: 0.9498898262212639


In [9]:


param_dist = {
    'C': loguniform(0.001, 10.0),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200]
}

grid_search = RandomizedSearchCV(
    estimator=LogisticRegression(),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая точность:", grid_search.best_score_)

best_model = grid_search.best_estimator_

Лучшие параметры: {'C': 1.1104304534907148, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'}
Лучшая точность: 0.9574835776773029


In [10]:


param_dist = {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'num_leaves': randint(15, 127),
    'min_child_samples': randint(5, 30),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

grid_search = RandomizedSearchCV(
    estimator=LGBMClassifier(),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая точность:", grid_search.best_score_)

best_model = grid_search.best_estimator_

[LightGBM] [Info] Number of positive: 158986, number of negative: 148359
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077940 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8988
[LightGBM] [Info] Number of data points in the train set: 307345, number of used features: 201
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.517288 -> initscore=0.069181
[LightGBM] [Info] Start training from score 0.069181
Лучшие параметры: {'learning_rate': 0.21145840183787665, 'max_depth': 7, 'min_child_samples': 22, 'n_estimators': 297, 'num_leaves': 48, 'reg_alpha': 0.919103162095239, 'reg_lambda': 0.5219330094767534}
Лучшая точность: 0.9661012566552746


In [11]:
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 0.5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

random_search = RandomizedSearchCV(
    estimator=XGBClassifier(),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train, y_train)

print("Лучшие параметры:", random_search.best_params_)
print("Лучшая точность:", random_search.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Лучшие параметры: {'colsample_bytree': 0.6727299868828402, 'gamma': 0.09170225492671691, 'learning_rate': 0.1012726728878613, 'max_depth': 8, 'n_estimators': 285, 'reg_alpha': 0.023062425041415757, 'reg_lambda': 0.5247746602583891, 'subsample': 0.7599443886861021}
Лучшая точность: 0.9658787046514524


In [12]:
param_dist = {
    'iterations': randint(100, 1000),
    'depth': randint(4, 10),
    'learning_rate': uniform(0.01, 0.3),
    'l2_leaf_reg': randint(1, 10),
    'border_count': [32, 64, 128, 256],
    'random_strength': uniform(0, 1)
}

random_search = RandomizedSearchCV(
    estimator=CatBoostClassifier(),
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print("Лучшие параметры:", random_search.best_params_)
print("Лучшая точность:", random_search.best_score_)

0:	learn: 0.6063976	total: 225ms	remaining: 2m 37s
1:	learn: 0.5424901	total: 306ms	remaining: 1m 46s
2:	learn: 0.4959275	total: 392ms	remaining: 1m 31s
3:	learn: 0.4625767	total: 477ms	remaining: 1m 22s
4:	learn: 0.4361183	total: 560ms	remaining: 1m 17s
5:	learn: 0.4151951	total: 638ms	remaining: 1m 13s
6:	learn: 0.3989537	total: 710ms	remaining: 1m 10s
7:	learn: 0.3852578	total: 786ms	remaining: 1m 8s
8:	learn: 0.3732725	total: 858ms	remaining: 1m 5s
9:	learn: 0.3644372	total: 928ms	remaining: 1m 4s
10:	learn: 0.3566525	total: 1s	remaining: 1m 2s
11:	learn: 0.3497417	total: 1.08s	remaining: 1m 1s
12:	learn: 0.3436685	total: 1.15s	remaining: 1m
13:	learn: 0.3387499	total: 1.22s	remaining: 60s
14:	learn: 0.3345981	total: 1.29s	remaining: 59.1s
15:	learn: 0.3308259	total: 1.37s	remaining: 58.4s
16:	learn: 0.3273746	total: 1.44s	remaining: 57.7s
17:	learn: 0.3244300	total: 1.5s	remaining: 56.8s
18:	learn: 0.3213656	total: 1.56s	remaining: 56s
19:	learn: 0.3188774	total: 1.63s	remaining: 

In [13]:
p16 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', VotingClassifier([
        ('lg', LogisticRegression(C= 1.1104304534907148, max_iter= 200, penalty= 'l1', solver= 'liblinear', random_state=42)),
        ('rf', RandomForestClassifier(max_depth= 14, min_samples_split= 5, n_estimators= 182, class_weight='balanced')),
        ('xgb', XGBClassifier(colsample_bytree= 0.6727299868828402, gamma= 0.09170225492671691, learning_rate= 0.1012726728878613, max_depth= 8, n_estimators= 285, reg_alpha= 0.023062425041415757, reg_lambda= 0.5247746602583891, subsample= 0.7599443886861021))
    ]))
])
p16_ = Pipeline([
    ('scaler', StandardScaler()),
    ('model', VotingClassifier([
        ('lg', LogisticRegression(C= 1.1104304534907148, max_iter= 200, penalty= 'l1', solver= 'liblinear', random_state=42)),
        ('xgb', XGBClassifier(colsample_bytree= 0.6727299868828402, gamma= 0.09170225492671691, learning_rate= 0.1012726728878613, max_depth= 8, n_estimators= 285, reg_alpha= 0.023062425041415757, reg_lambda= 0.5247746602583891, subsample= 0.7599443886861021))
    ]))
])

In [14]:
for i,p in enumerate([p16, p16_]):
    p.fit(X_train, y_train)
    pred = p.predict(X_test)
    print(i+1, roc_auc_score(y_test, pred))

1 0.8935569760779377
2 0.8948817144128488


In [15]:
p17 = Pipeline([
    ('model', RandomForestClassifier(max_depth= 14, min_samples_split= 5, n_estimators= 182, class_weight='balanced'))
])
p18 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(C= 1.1104304534907148, max_iter= 200, penalty= 'l1', solver= 'liblinear', random_state=42))
])
p19 = Pipeline([
    ('model', XGBClassifier(colsample_bytree= 0.6727299868828402, gamma= 0.09170225492671691, learning_rate= 0.1012726728878613, max_depth= 8, n_estimators= 285, reg_alpha= 0.023062425041415757, reg_lambda= 0.5247746602583891, subsample= 0.7599443886861021))
])
p20 = Pipeline([
    ('model', CatBoostClassifier(border_count= 64, depth= 7, iterations= 700, l2_leaf_reg= 2, learning_rate= 0.12562495076197483, random_strength= 0.015966252220214194))
])
p21 = Pipeline([
    ('model', LGBMClassifier(learning_rate= 0.21145840183787665, max_depth= 7, min_child_samples= 22, n_estimators= 297, num_leaves= 48, reg_alpha= 0.919103162095239, reg_lambda= 0.5219330094767534))
])

In [17]:
for i,p in enumerate([p16, p16_, p17, p18, p19, p20, p21]):
    p.fit(X_train, y_train)
    pred = p.predict(X_test)
    print(i+1, roc_auc_score(y_test, pred))

1 0.8940022455109455
2 0.8948817144128488
3 0.871055754746852
4 0.8937079551308353
5 0.8943509785752486
0:	learn: 0.6063976	total: 82.5ms	remaining: 57.7s
1:	learn: 0.5424901	total: 163ms	remaining: 57s
2:	learn: 0.4959275	total: 245ms	remaining: 57s
3:	learn: 0.4625767	total: 326ms	remaining: 56.7s
4:	learn: 0.4361183	total: 402ms	remaining: 55.8s
5:	learn: 0.4151951	total: 478ms	remaining: 55.3s
6:	learn: 0.3989537	total: 547ms	remaining: 54.1s
7:	learn: 0.3852578	total: 619ms	remaining: 53.6s
8:	learn: 0.3732725	total: 689ms	remaining: 52.9s
9:	learn: 0.3644372	total: 754ms	remaining: 52s
10:	learn: 0.3566525	total: 824ms	remaining: 51.6s
11:	learn: 0.3497417	total: 891ms	remaining: 51.1s
12:	learn: 0.3436685	total: 958ms	remaining: 50.6s
13:	learn: 0.3387499	total: 1.02s	remaining: 50.1s
14:	learn: 0.3345981	total: 1.09s	remaining: 49.7s
15:	learn: 0.3308259	total: 1.16s	remaining: 49.4s
16:	learn: 0.3273746	total: 1.22s	remaining: 49.1s
17:	learn: 0.3244300	total: 1.28s	remaining:

In [18]:
model = CatBoostClassifier(border_count= 64, depth= 7, iterations= 700, l2_leaf_reg= 2, learning_rate= 0.12562495076197483, random_strength= 0.015966252220214194)
model.fit(X, y)
y_proba = model.predict_proba(df_test)

0:	learn: 0.6063638	total: 275ms	remaining: 3m 11s
1:	learn: 0.5421110	total: 513ms	remaining: 2m 59s
2:	learn: 0.4962669	total: 619ms	remaining: 2m 23s
3:	learn: 0.4619496	total: 713ms	remaining: 2m 4s
4:	learn: 0.4357694	total: 807ms	remaining: 1m 52s
5:	learn: 0.4145843	total: 898ms	remaining: 1m 43s
6:	learn: 0.3982566	total: 985ms	remaining: 1m 37s
7:	learn: 0.3846718	total: 1.07s	remaining: 1m 32s
8:	learn: 0.3735668	total: 1.16s	remaining: 1m 29s
9:	learn: 0.3646795	total: 1.24s	remaining: 1m 25s
10:	learn: 0.3563840	total: 1.32s	remaining: 1m 22s
11:	learn: 0.3496526	total: 1.42s	remaining: 1m 21s
12:	learn: 0.3436600	total: 1.5s	remaining: 1m 19s
13:	learn: 0.3385694	total: 1.58s	remaining: 1m 17s
14:	learn: 0.3329818	total: 1.67s	remaining: 1m 16s
15:	learn: 0.3293185	total: 1.75s	remaining: 1m 14s
16:	learn: 0.3260693	total: 1.82s	remaining: 1m 13s
17:	learn: 0.3230479	total: 1.9s	remaining: 1m 12s
18:	learn: 0.3203638	total: 1.98s	remaining: 1m 10s
19:	learn: 0.3175003	tota

In [19]:

presub = pd.DataFrame(y_proba)
sub = pd.concat([match_id, presub[1]], axis=1)
sub = sub.rename(columns={1: 'radiant_win'})

final_sub = sub.head(31946)
print(final_sub)
final_sub.to_csv('DOTA2.csv', index=False)

       match_id  radiant_win
0             1     0.836180
1             2     0.961413
2             3     0.000628
3             4     0.999799
4             5     0.236145
...         ...          ...
31941     31942     0.000026
31942     31943     0.004275
31943     31944     0.106512
31944     31945     0.000061
31945     31946     0.034841

[31946 rows x 2 columns]
